In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import os

In [ ]:
root = "/Users/nick/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/morphseq/"
metadata_df = pd.read_csv(os.path.join(root, "metadata", "combined_metadata_files", "embryo_metadata_df01.csv"))
metadata_df = metadata_df.loc[metadata_df["use_embryo_flag"]==1, :]
np.mean(metadata_df["use_embryo_flag"])

In [ ]:
np.unique(metadata_df["experiment_date"].astype(str))

### Look at embryo length vs. predicted stage

In [ ]:
sample_date = "20231206"
metadata_df["experiment_date"] = metadata_df["experiment_date"].astype(str)
ft = (metadata_df["experiment_date"]==sample_date)# & (metadata_df["embryo_id"]=="20230531_D03_e00")
fig = px.scatter(metadata_df.loc[ft, :], 
                 x="predicted_stage_hpf", y="surface_area_um", color="use_embryo_flag", hover_data={"snip_id"})
fig.show()

### Write outline for stage calibration function that uses embryo length

#### Build reference key

In [ ]:
ref_date01 = "20230620"
stage_df01 = metadata_df.loc[metadata_df["experiment_date"]==ref_date01, ["snip_id", "embryo_id", "predicted_stage_hpf", "surface_area_um", "length_um", "use_embryo_flag"]].reset_index(drop=True)
# ref_bool = (stage_df.loc[:, "phenotype"].to_numpy() == "wt") | (stage_df.loc[:, "control_flag"].to_numpy() == 1)
# ref_bool = ref_bool | (stage_df.loc[:, "phenotype"].to_numpy() == "uncertain")
# ref_bool = ref_bool & stage_df["use_embryo_flag"]
# stage_df = stage_df.loc[ref_bool]
stage_df01["stage_group_hpf"] = np.round(stage_df01["predicted_stage_hpf"])
stage_key_df01 = stage_df01.groupby('stage_group_hpf').quantile(.90).reset_index()

ref_date02 = "20240626"
stage_df02 = metadata_df.loc[metadata_df["experiment_date"]==ref_date02, ["snip_id", "embryo_id", "predicted_stage_hpf", "surface_area_um", "length_um", "use_embryo_flag"]].reset_index(drop=True)
stage_df02["stage_group_hpf"] = np.round(stage_df02["predicted_stage_hpf"])
stage_key_df02 = stage_df02.groupby('stage_group_hpf').quantile(.90).reset_index()

stage_key_df = pd.concat([stage_key_df01.loc[stage_key_df01["predicted_stage_hpf"] <= 12, :], stage_key_df02], axis=0, ignore_index=True)

fig = px.scatter(stage_key_df, x="stage_group_hpf", y="surface_area_um")
fig.show()

In [ ]:
import scipy

# A vs time
ref_time_vec = stage_key_df["stage_group_hpf"]
ref_sa_vec = stage_key_df["surface_area_um"]


def sigmoid(params, t_vec=ref_time_vec):
    sa_pd = params[0] + params[1]* np.divide(t_vec**params[2], params[3]**params[2] + t_vec**params[2])
    return sa_pd

# define loss
def loss_fun(params, sa=ref_sa_vec):
    loss = sigmoid(params) - ref_sa_vec
    return loss

# def height_fun(params, repo_areas=repo_lengths[boot_indices]):
#         h = np.divide(params[0] * repo_areas**params[2], (params[1]**params[2] + repo_areas**params[2]))
#         return h

# def loss_fun(params, repo_heights=repo_heights[boot_indices]):
#     h_hat = height_fun(params)
#     return repo_heights - h_hat

x0 = [3e5, 1.9e6, 2, 24] 
# sigmoid(x0)
params_fit = scipy.optimize.least_squares(loss_fun, x0, bounds=[(0, 0, 0, 0), (np.inf, np.inf, np.inf, np.inf)])

print(params_fit)
full_time = np.linspace(0, 96)
sa_pd = sigmoid(params_fit.x, t_vec=full_time)

fig = px.scatter(stage_key_df, x="stage_group_hpf", y="surface_area_um")
fig.add_trace(go.Scatter(x=full_time, y=sa_pd, mode="lines"))
fig.show()


In [ ]:
# save 
stage_ref_df = pd.DataFrame(full_time, columns=["stage_hpf"])
stage_ref_df["sa_um"] = sa_pd
stage_ref_df.to_csv(os.path.join(root, "metadata", "stage_ref_df.csv"), index=False)

param_df = pd.DataFrame([params_fit.x], columns=["offset", "sa_max", "hill_coeff", "inflection_point"])
param_df.to_csv(os.path.join(root, "metadata", "stage_ref_params.csv"), index=False)

param_df.head()

In [ ]:
def t_from_sa(params, sa_vec):
    t_pd = params[3] * np.divide(sa_vec-params[0], params[1] - sa_vec + params[0])**(1/params[2])
    return t_pd

t_test = t_from_sa(params_fit.x, sa_pd)

px.scatter(x=full_time, y=t_test)